In [10]:
nnnn = {}
with open('real_results/PAKDD_neighbors.txt', 'r') as file:
    cnt = 0
    lines = file.readlines()
    for i,l in enumerate(lines):
        cnt += 1
        n = []
        arr = l.split(" ")
        for item in arr:

            try:
                n.append(int(item))

            except:
                try:
                    n.append(int(item[1:]))

                except:
                    try:
                        n.append(int(item[:-2]))

                    except:
                        None
        if len(n) != 6:
            print(f"line:{i} cnt:{len(n)}")
        nnnn[n[0]] = n[1:]
    print(len(nnnn))
    print(cnt)




9874
9874


In [2]:
import pandas as pd
import numpy as np

In [11]:
df = pd.read_csv("dataset/PAKDD.csv",header=None,index_col=False)
df

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,22,F,O,44,0,N,31,12,0,P,...,Y,N,0,N,N,300.0,0,0,N,0
1,15,F,S,18,0,Y,31,20,0,P,...,Y,N,0,N,N,300.0,0,0,N,0
2,24,F,C,22,0,Y,31,8,0,P,...,Y,N,0,N,N,229.0,0,0,N,0
3,12,F,C,47,0,N,31,25,0,P,...,Y,N,0,N,N,304.0,0,0,N,0
4,16,F,S,28,0,Y,31,25,0,O,...,Y,N,0,N,N,250.0,0,0,N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49995,19,M,S,21,0,Y,31,12,0,P,...,Y,N,0,N,N,234.0,0,0,N,0
49996,22,F,C,40,0,Y,31,1,0,P,...,Y,N,0,N,N,1500.0,0,0,N,0
49997,23,F,S,23,0,Y,31,28,0,P,...,Y,N,0,N,N,240.0,0,0,N,1
49998,9,F,C,38,0,Y,5,28,0,P,...,Y,N,0,N,N,616.0,0,0,N,0


In [12]:
Y = df[27]
majorityClass = np.bincount(Y).argmax()

In [13]:
df[Y != majorityClass]

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
8,3,F,S,27,0,Y,31,20,0,A,...,Y,N,0,N,N,1000.0,0,0,N,1
10,25,F,S,53,0,Y,31,18,0,P,...,Y,N,0,N,N,738.0,0,1,N,1
19,24,M,S,20,0,Y,31,28,0,P,...,Y,N,0,N,N,500.0,0,0,N,1
20,9,M,C,26,0,Y,5,18,0,C,...,Y,N,0,N,N,490.0,0,0,N,1
21,1,M,C,17,0,N,50,12,0,A,...,Y,N,0,N,N,426.0,0,0,N,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49955,22,M,S,18,0,Y,31,20,0,P,...,Y,N,0,N,N,400.0,0,1,N,1
49967,19,F,C,23,0,Y,31,20,0,A,...,Y,N,0,N,N,600.0,0,0,N,1
49990,15,M,C,43,0,Y,31,28,0,P,...,Y,N,0,N,N,3500.0,0,1,N,1
49993,1,M,C,29,0,Y,31,12,0,A,...,Y,N,0,N,N,796.0,0,1,N,1


In [14]:
results = pd.DataFrame(columns = ["i","class", "category"])
for item, nearests in nnnn.items():
    cl = Y[item]
    same = 0
    other = 0
    category = "safe"
    for nn in nearests:
        if Y[nn] == cl:
            same += 1
        else:
            other += 1
    if same == 1:
        category = "outlier"
    elif same <= 2:
        category = "rare"
    elif same <= 4:
        category = "borderline"

    df2 = pd.DataFrame([[item,cl,category]],columns = ["i","class", "category"])

    results = results.append(df2)
results

,i,class,category
0,8,1,outlier
0,10,1,outlier
0,19,1,outlier
0,20,1,outlier
0,21,1,borderline
...,...,...,...
0,49955,1,rare
0,49967,1,rare
0,49990,1,rare
0,49993,1,outlier


In [15]:
def map_p(i):
    divider = zeros
    if categories_counts["class"][i] != majorityClass:
        divider = ones
    return int(categories_counts["i"][i] * 10000/divider) / 100

In [16]:
zeros = results[results["class"]==majorityClass].count()[0]
ones = results[results["class"]!= majorityClass].count()[0]
categories_counts = results.groupby(["class","category"]).count()
categories_counts = categories_counts.reset_index()
categories_counts["p"] = pd.Series(categories_counts.index).apply(map_p)
categories_counts

,class,category,i,p
0,0,borderline,4,100.00
1,1,borderline,1425,14.43
2,1,outlier,3325,33.68
3,1,rare,5047,51.13
4,1,safe,73,0.73


In [17]:
categories_counts.to_csv("PAKDD_results.csv", header=False)